# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# perhaps add on 'weather_delay' if we have time
columns = ['fl_date', 'mkt_carrier', 'origin', 'dest', 'distance', 'arr_delay']
data = pd.read_csv('data/airline_sample_150k.csv', usecols=columns)

In [3]:
data.head()

,fl_date,mkt_carrier,origin,dest,arr_delay,distance
0,2019-01-01,AA,CLT,JAN,29.0,562
1,2019-01-01,NK,BWI,DEN,15.0,1491
2,2019-01-01,AA,DTW,ORD,94.0,235
3,2019-01-01,AA,PIB,MEI,-15.0,69
4,2019-01-01,DL,LAN,DTW,-7.0,74


### Data Transformation

In [4]:
# transform categorical features into dummies

cat_cols = ['mkt_carrier', 'origin', 'dest']
cols_dummies = pd.get_dummies(data[cat_cols])

In [5]:
cols_dummies

,mkt_carrier_AA,mkt_carrier_AS,mkt_carrier_B6,mkt_carrier_DL,mkt_carrier_F9,mkt_carrier_G4,mkt_carrier_HA,mkt_carrier_NK,mkt_carrier_UA,mkt_carrier_VX,...,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158816,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158817,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158818,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158819,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data = data.drop(cat_cols, axis=1)

In [7]:
data

,fl_date,arr_delay,distance
0,2019-01-01,29.0,562
1,2019-01-01,15.0,1491
2,2019-01-01,94.0,235
3,2019-01-01,-15.0,69
4,2019-01-01,-7.0,74
...,...,...,...
158816,2019-09-30,16.0,184
158817,2019-09-30,-30.0,1076
158818,2019-09-30,27.0,1065
158819,2019-09-30,-21.0,231


In [8]:
data_tf = pd.concat([data, cols_dummies], axis=1)

In [9]:
data.shape

(158821, 3)

In [10]:
data.isnull().sum()

fl_date         0
arr_delay    3064
distance        0
dtype: int64

In [11]:
data_tf = data_tf.dropna().reset_index(drop=True)

In [12]:
data_tf

,fl_date,arr_delay,distance,mkt_carrier_AA,mkt_carrier_AS,mkt_carrier_B6,mkt_carrier_DL,mkt_carrier_F9,mkt_carrier_G4,mkt_carrier_HA,...,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM
0,2019-01-01,29.0,562,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-01,15.0,1491,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019-01-01,94.0,235,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019-01-01,-15.0,69,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-01-01,-7.0,74,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155752,2019-09-30,16.0,184,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155753,2019-09-30,-30.0,1076,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155754,2019-09-30,27.0,1065,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155755,2019-09-30,-21.0,231,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# transform fl_date into only month

data_tf['months'] = [int(data_tf['fl_date'][i].split('-')[1]) for i in range(len(data_tf))]
data_tf = data_tf.drop('fl_date', axis=1)

In [14]:
data_tf

,arr_delay,distance,mkt_carrier_AA,mkt_carrier_AS,mkt_carrier_B6,mkt_carrier_DL,mkt_carrier_F9,mkt_carrier_G4,mkt_carrier_HA,mkt_carrier_NK,...,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM,months
0,29.0,562,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,15.0,1491,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,94.0,235,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-15.0,69,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,-7.0,74,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155752,16.0,184,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
155753,-30.0,1076,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
155754,27.0,1065,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
155755,-21.0,231,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [15]:
# data_tf = data_tf.groupby('months', as_index=False).mean()

In [16]:
# data_tf

### Split Training and Testing

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = data_tf.drop('arr_delay', axis=1)
const = np.ones(len(data_tf))
X['const'] = const

y = data_tf.arr_delay
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

### Linear Regression Fit and Predict

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [20]:
model = LinearRegression()
selector_to_fit = RFE(estimator=model, n_features_to_select=10)
selector = selector_to_fit.fit(X_train, y_train)

In [22]:
y_pred = selector.predict(X_test)

### RMSE Test

In [24]:
from sklearn import metrics

In [28]:
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

49.171069616691916


In [31]:
print(metrics.r2_score(y_test, y_pred))

0.003869416431459771


#### Evaluation Test

In [30]:
data_test = pd.read_csv('data/flight_test_2days.csv')

In [32]:
data_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,1577865600000,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1577865600000,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,1577865600000,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,1577865600000,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,1577865600000,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42144,1577865600000,UA,UA_CODESHARE,UA,4383,EV,N34110,4383,10693,BNA,"Nashville, TN",12266,IAH,"Houston, TX",1442,1717,N,155,1,657
42145,1577865600000,UA,UA_CODESHARE,UA,4384,EV,N11192,4384,12266,IAH,"Houston, TX",11193,CVG,"Cincinnati, OH",1215,1542,N,147,1,871
42146,1577865600000,UA,UA_CODESHARE,UA,4385,EV,N16559,4385,12266,IAH,"Houston, TX",15412,TYS,"Knoxville, TN",1942,2255,N,133,1,771
42147,1577865600000,UA,UA_CODESHARE,UA,4387,EV,N13992,4387,12266,IAH,"Houston, TX",10747,BRO,"Brownsville, TX",1420,1546,N,86,1,308


## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.